# Семинар 1

В рамках сегодняшнего семинара мы с вами посмотрим на различные методы Parameter Efficient Fine-Tuning`а, релизуем некоторые на чистом pytorch, а также посмотрим на удобную бублиотеку peft от huggingface, которая позвволяет буквально в одну строчку интегрировать множесто PEFT методов вашу модель.

Начнем с самого часто используемого peft-метода - [LoRA](https://arxiv.org/abs/2106.09685).

## LoRA

LoRA является одной из наиболее широко используемых и эффективных техник для эффективного обучения LLM. Это очень важная метод, который используется сейчас повсеместно, поэтому мы разберем его подробно

Поскольку большие языковые модели тяжелые, обновление всех весов модели во время обучения может быть дорогостоящим из-за ограничений памяти GPU. Предположим, у нас есть большая матрица весов W для данного слоя. Во время обратного распространения ошибки мы учимся матрице ΔW, которая содержит информацию о том, насколько мы хотим обновить исходные веса, чтобы минимизировать функцию потерь во время обучения.

В обычном режиме обучения обновление весов определяется следующим образом:

$$W_{updated}=W+\Delta W$$

LoRA предлагает более эффективную альтернативу вычислению обновлений весов ΔW за счет обучения их приближенного представления в виде произведения двух **низкоранговых** матриц А и В, ΔW ≈ AB:

$$W_{updated}=W+A*B$$

![](./images/lora_1.webp)

Как LoRA экономит память GPU?   
Если предобученная матрица весов W является матрицей размером 1,000×1,000, то матрица обновления весов ΔW при обычном fine-tune также будет матрицей 1,000×1,000. В этом случае ΔW имеет 1,000,000 параметров.  
Если мы рассмотрим LoRA ранга 2, то A будет матрицей 1000×2, а B - матрицей 2×1000, и нам нужно обновить только 2×2×1,000 = 4,000 параметров при использовании LoRA. В предыдущем примере, с рангом 2, это **в 250 раз меньше параметров**.

Конечно, A и B не могут отражать всю информацию, которую есть в ΔW, но это сделано намеренно. Используя LoRA, мы предполагаем, что модели требуется, чтобы W была большой матрицей полного ранга, чтобы инкапсулировать все знание, накопленное в результате предобучения модели.  
Однако во время fine-tun`а LLM, нам не нужно обновлять все веса и сильно менять накопленную информацию - достаточно приближенного обновления, которое достигается апроксимацией исходного обнолвения в более низкогоранговом предситавлении, которое мы формулируем через матрицы AB.

Обратите внимание, что описанное выше немного расходится с тем, что изображено на рисунке выше. Это связано с распределительным законом умножения матриц: нам не нужно добавлять веса к обновляемым весами, но мы можем держать их отдельно. Например, если x - входные данные, тогда мы можем написать следующее для обычного fine-tuning`а:

$$x.(W+\Delta W)=x.W+x.\Delta W$$

или тоже самое для LoRA:

$$x.(W+A*B)=x.W+x*A*B$$

Тот факт, что мы можем держать матрицы весов LoRA отдельно, делает LoRA особенно привлекательным. На практике это означает, что нам вообще не нужно изменять веса предобученной модели, так как мы можем применять матрицы LoRA на этапе inference. Это особенно полезно, если вы рассматриваете возможность хостинга модели для нескольких клиентов. Вместо того, чтобы сохранять большие обновленные модели для каждого клиента, вам нужно сохранить только небольшой набор весов LoRA вместе с оригинальной предобученной моделью.

Простая реализация этого механизма выглядит очень просто в коде:

In [1]:
import torch
import torch.nn as nn

if torch.cuda.is_available():
    # NVIDIA CUDA Deep Neural Network (cuDNN) is a GPU-accelerated library of primitives for deep neural networks
    torch.backends.cudnn.deterministic=True


class LoRALayer(nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super().__init__()
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())
        self.A = nn.Parameter(torch.randn(in_dim, rank) * std_dev)
        self.B = nn.Parameter(torch.zeros(rank, out_dim))
        self.alpha = alpha

    def forward(self, x):
        x = self.alpha * (x @ self.A @ self.B)
        return x

Обратите внимание, что у LoRA есть два гиперпараметра - `rank` и `alpha`. `rank` является гиперпараметром, который контролирует внутреннее измерение матриц A и B. Другими словами, этот параметр контролирует количество дополнительных параметров, введенных LoRA, и является ключевым фактором в определении баланса между адаптируемостью модели и эффективностью параметров.

Второй гиперпараметр, `alpha`, является масштабирующим гиперпараметром, применяемым к выходу низкогорангового представления матрицы. Он по сути контролирует степень, в которой допускается влияние выхода адаптированного слоя на оригинальный выход адаптируемого слоя. Это может рассматриваться как способ регулирования влияния адаптации низкого ранга на выход исходного слоя.

Чтобы заменить имеющийся слой на какой либо текущий необходимо просто добавить реализацию выше к готовому линейному слою

In [124]:
class LinearWithLoRA(nn.Module):
    def __init__(self, linear, rank, alpha, device='cuda'):
        super().__init__()
        self.linear=linear
        self.lora=LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        ).to(device)
    
    def forward(self, x):
        return self.linear(x)+self.lora(x)

Попробуем сравнить обучение полной модели, и с использование peft-методов.
Возьмем готовый код из предыдущего семинара по обучению модели BERT и посмотрим на разницу в скорости обучения и качеству 

In [3]:
!wget https://raw.githubusercontent.com/snv-ds/NLP_course/master/week2/train -q

In [30]:
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
from lightning import Fabric
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-conversational")

def prepare_data():
    train = pd.read_csv('train', sep='\t', names=['id','target','temp1','temp2','comment'], index_col=0)
    parse_labels = ['__label__NORMAL','__label__INSULT','__label__THREAT','__label__OBSCENITY']
    
    mask = train['comment'].isin(parse_labels) # to cope only with correct rows in data
    
    train.loc[mask,'target'] = train[mask]['target'] + ',' + train[mask]['comment']
    train.loc[mask,'comment'] = np.nan
    
    for t in ['temp1','temp2']: # if comment have several labels of classes
        mask = train[t].isin(parse_labels)
        train.loc[mask,'target'] = train[mask]['target'] + ',' + train[mask][t]
        train.loc[mask,t] = np.nan
        train.loc[~train[t].isna(),'comment'] = train[~train[t].isna()][t]
    
    train[['оскорбление','другое','непростойность','угроза']] = train['target'].str.get_dummies(sep=',')
    
    train = train[['другое','оскорбление','непростойность','угроза', 'comment']]
    return train

def split_data_for_training(df, test_size=0.2):
    train_text, val_text, train_labels, val_labels = train_test_split(df.comment,
                                                                      df[['другое','оскорбление',
                                                                         'непростойность','угроза']],
                                                                      test_size = test_size,
                                                                      random_state=2029)
    return train_text, val_text, train_labels, val_labels

def prepare_tokens(tokenizer, text, max_len=50):
    tokens = tokenizer.batch_encode_plus(
        text.tolist(),
        max_length = max_len,
        padding='max_length',
        truncation=True
    )
    return tokens

def init_data_loaders(text: pd.Series, labels: pd.Series, batch_size=32, tokenizer=tokenizer):
    tokenized_text = prepare_tokens(tokenizer, text)
    input_ids = torch.tensor(tokenized_text['input_ids'])
    mask = torch.tensor(tokenized_text['attention_mask'])
    labels = torch.tensor(labels.values.tolist(), dtype=torch.float)

    dataset = TensorDataset(input_ids, mask, labels)
    sampler = RandomSampler(dataset)
    dataloader = DataLoader(dataset, sampler=sampler, batch_size=batch_size)
    return dataloader

In [74]:
id2label = dict(enumerate(['__label__NORMAL','__label__INSULT','__label__THREAT','__label__OBSCENITY']))
label2id = {label: ind for ind, label in id2label.items()}

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_embeder = AutoModelForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased-conversational", num_labels=len(id2label), id2label=id2label, label2id=label2id
).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [75]:
train_text, val_text, train_labels, val_labels = split_data_for_training(prepare_data())
train_dataloader, val_dataloader = init_data_loaders(train_text, train_labels), init_data_loaders(val_text, val_labels)

# model_embeder = BERT_Arch(model, n_output=4).to(device)
print(sum(param.numel() for param in model_embeder.parameters()), " parameters in model")
optimizer = AdamW(model_embeder.parameters(), lr = 1e-5)
cross_entropy = torch.nn.functional.binary_cross_entropy_with_logits
epochs = 5

177856516  parameters in model


In [76]:
def train(model):
    model.train()
    total_loss = 0
    total_preds=[]
  
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = [r.to(device) for r in batch]
        sent_id, mask, labels = batch
    
        model.zero_grad()        
        preds = model(sent_id, mask).logits
        loss = cross_entropy(preds, labels)
        total_loss = total_loss + loss.item()
      
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_preds.append(preds.detach().cpu().numpy())

    avg_loss = total_loss / len(train_dataloader)
    total_preds  = np.concatenate(total_preds, axis=0)
    return (avg_loss, total_preds), model

In [77]:
def evaluate(model):
    print("\nEvaluating...")  
    model.eval()

    total_loss = 0
    total_preds, total_labels = [], []
  
    for step,batch in enumerate(tqdm(val_dataloader)):
        batch = [t.to(device) for t in batch]
        sent_id, mask, labels = batch

        with torch.no_grad():
            preds = model(sent_id, mask).logits
            loss = cross_entropy(preds,labels)
            total_loss = total_loss + loss.item()
    
            total_preds.append(preds.detach().cpu().numpy())
            total_labels.append(labels.detach().cpu().numpy())

    avg_loss = total_loss / len(val_dataloader) 

    total_preds  = np.concatenate(total_preds, axis=0)
    total_labels  = np.concatenate(total_labels, axis=0)
  
    print(classification_report(total_labels,
                                torch.sigmoid(torch.tensor(total_preds)).round(),
                                zero_division=True))

    return (avg_loss, total_preds),  model

In [78]:
best_valid_loss = float('inf')

train_losses=[]
valid_losses=[]

for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    (train_loss, _), model_embeder = train(model_embeder)
    (valid_loss, _), model_embeder = evaluate(model_embeder)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model_embeder.state_dict(), 'saved_weights.pt')
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 5


  0%|          | 0/3720 [00:00<?, ?it/s]


Evaluating...


  0%|          | 0/930 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.99      0.98      0.99     24352
           1       0.90      0.93      0.92      4458
           2       0.81      0.81      0.81       470
           3       0.80      0.92      0.86      1507

   micro avg       0.96      0.97      0.97     30787
   macro avg       0.88      0.91      0.89     30787
weighted avg       0.97      0.97      0.97     30787
 samples avg       0.97      0.97      0.97     30787


Training Loss: 0.071
Validation Loss: 0.054

 Epoch 2 / 5


...

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     24352
           1       0.93      0.90      0.91      4458
           2       0.80      0.88      0.84       470
           3       0.88      0.88      0.88      1507

   micro avg       0.97      0.97      0.97     30787
   macro avg       0.90      0.91      0.90     30787
weighted avg       0.97      0.97      0.97     30787
 samples avg       0.97      0.97      0.97     30787


Training Loss: 0.014
Validation Loss: 0.074

 Epoch 5 / 5


  0%|          | 0/3720 [00:00<?, ?it/s]


Evaluating...


  0%|          | 0/930 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     24352
           1       0.91      0.92      0.91      4458
           2       0.81      0.85      0.83       470
           3       0.87      0.89      0.88      1507

   micro avg       0.97      0.97      0.97     30787
   macro avg       0.89      0.91      0.90     30787
weighted avg       0.97      0.97      0.97     30787
 samples avg       0.97      0.97      0.97     30787


Training Loss: 0.010
Validation Loss: 0.092


Как выидите  получилось отличное качество модели, но при этом мы обучали все 177М параметров модели. Попробуем альтернативные варианты

In [55]:
print(sum(param.numel() for param in model_embeder.parameters() if param.requires_grad), "trainable parameters in model")

177856516 trainable parameters in model


In [173]:
from copy import deepcopy


model_embeder = AutoModelForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased-conversational", num_labels=len(id2label), id2label=id2label, label2id=label2id
).to(device)
lora_model_embeder = deepcopy(model_embeder).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Заморозим веса исходной модели, и попробуем обучить теперь LoRA-адаптеры для нашей модели

In [174]:
for param in lora_model_embeder.bert.parameters():
    param.requires_grad = False
print(sum(param.numel() for param in lora_model_embeder.parameters() if param.requires_grad), "trainable parameters in model")

3076 trainable parameters in model


In [175]:
for name, param in lora_model_embeder.bert.named_parameters():
    print(f"{name}: {param.requires_grad}")

embeddings.word_embeddings.weight: False
embeddings.position_embeddings.weight: False
embeddings.token_type_embeddings.weight: False
embeddings.LayerNorm.weight: False
embeddings.LayerNorm.bias: False
encoder.layer.0.attention.self.query.weight: False
encoder.layer.0.attention.self.query.bias: False
encoder.layer.0.attention.self.key.weight: False
encoder.layer.0.attention.self.key.bias: False
encoder.layer.0.attention.self.value.weight: False
encoder.layer.0.attention.self.value.bias: False
encoder.layer.0.attention.output.dense.weight: False
encoder.layer.0.attention.output.dense.bias: False
encoder.layer.0.attention.output.LayerNorm.weight: False
encoder.layer.0.attention.output.LayerNorm.bias: False
encoder.layer.0.intermediate.dense.weight: False
encoder.layer.0.intermediate.dense.bias: False
encoder.layer.0.output.dense.weight: False
encoder.layer.0.output.dense.bias: False
encoder.layer.0.output.LayerNorm.weight: False
encoder.layer.0.output.LayerNorm.bias: False
encoder.layer.1

Оригинальная статья адаптирует только веса слоя attention в Transformer с помощью LoRA. adapter-transformers дополнительно позволяет вводить LoRA в FFN слои блока Transformer.

добавим LoRA в каждый слой attention`а. Самостоятельно вы можете попробовать и другие слои для добавления

In [176]:
for i in range(12):  # N of layers in BERT
        lora_model_embeder.bert.encoder.layer[i].attention.self.query = LinearWithLoRA(
            lora_model_embeder.bert.encoder.layer[i].attention.self.query, 8, 32
        )
        lora_model_embeder.bert.encoder.layer[i].attention.self.key = LinearWithLoRA(
            lora_model_embeder.bert.encoder.layer[i].attention.self.key, 8, 32
        )
        lora_model_embeder.bert.encoder.layer[i].attention.self.value = LinearWithLoRA(
            lora_model_embeder.bert.encoder.layer[i].attention.self.value, 8, 32
        )
        lora_model_embeder.bert.encoder.layer[i].attention.output.dense = LinearWithLoRA(
            lora_model_embeder.bert.encoder.layer[i].attention.output.dense, 8, 32
        )
print(sum(param.numel() for param in lora_model_embeder.parameters() if param.requires_grad), "trainable parameters in model")

592900 trainable parameters in model


Получили всего около 600К обучаемых параметров, что заметно меньше исходного количества для полного FT

In [177]:
total_param_number = sum(param.numel() for param in model_embeder.parameters())
n_lora_trainable_params = sum(param.numel() for param in lora_model_embeder.parameters() if param.requires_grad)
print(f"{round((n_lora_trainable_params / 177856516) * 100, 3)}% of total amount of parameters")

0.333% of total amount of parameters


In [178]:
best_valid_loss = float('inf')

optimizer = AdamW(lora_model_embeder.parameters(), lr = 1e-5)
cross_entropy = torch.nn.functional.binary_cross_entropy_with_logits
epochs = 3

train_losses=[]
valid_losses=[]

for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    (train_loss, _), model_embeder = train(lora_model_embeder)
    (valid_loss, _), model_embeder = evaluate(lora_model_embeder)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(lora_model_embeder.state_dict(), 'saved_weights.pt')
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 3


  0%|          | 0/3720 [00:00<?, ?it/s]


Evaluating...


  0%|          | 0/930 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.99      0.98      0.98     24352
           1       0.87      0.91      0.89      4458
           2       0.60      0.67      0.64       470
           3       0.84      0.82      0.83      1507

   micro avg       0.95      0.95      0.95     30787
   macro avg       0.82      0.84      0.83     30787
weighted avg       0.96      0.95      0.95     30787
 samples avg       0.96      0.96      0.96     30787


Training Loss: 0.133
Validation Loss: 0.074

 Epoch 2 / 3


...

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     24352
           1       0.89      0.88      0.88      4458
           2       0.70      0.77      0.73       470
           3       0.81      0.83      0.82      1507

   micro avg       0.96      0.95      0.95     30787
   macro avg       0.84      0.86      0.85     30787
weighted avg       0.96      0.95      0.95     30787
 samples avg       0.96      0.96      0.96     30787


Training Loss: 0.083
Validation Loss: 0.068

 Epoch 3 / 3


  0%|          | 0/3720 [00:00<?, ?it/s]


Evaluating...


  0%|          | 0/930 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.99      0.96      0.98     24352
           1       0.81      0.94      0.87      4458
           2       0.60      0.79      0.68       470
           3       0.86      0.77      0.81      1507

   micro avg       0.94      0.95      0.95     30787
   macro avg       0.81      0.87      0.83     30787
weighted avg       0.95      0.95      0.95     30787
 samples avg       0.95      0.95      0.95     30787


Training Loss: 0.082
Validation Loss: 0.080


Как видите, этот метод достигает практически аналогичных значений качества при этом обучая всего 0.33% параметров модели, при этом затрачивая почти вдвое меньше времени

## IA3

Чтобы сделать Fine-tuning еще эффективнее, метод [IA^3](https://arxiv.org/pdf/2205.05638.pdf) (Infused Adapter by Inhibiting and Amplifying Inner Activations) масштабирует внутренние активации с помощью изученных векторов. Эти обучаемые вектора вводятся в модули attention и feedforward блоки в типовой трансформерной архитектуре. Эти обучаемые вектора являются единственными обучаемыми параметрами во время Fine-tuning`а, и таким образом оригинальные веса остаются замороженными. Обучая только эти вектора (в отличие от матриц в LoRA) еще больше уменьшает общее количество обучаемых параметров, что позволяет потенциально еще эффективнее обучать модель.

![](./images/IA\*\*3.png)

(IA)^3 вводит обучаемые векторы $l_{W}$ в различные компоненты модели Transformer, которые выполняют поэлементное масштабирование внутренних активаций модели. Таким образом, для любого слоя модели, выраженного как матричное умножение вида h=Wx, он выполняет поэлементное умножение с $l_{W}$, токое что:

$$h=l_{W}⊙Wx$$

Пример псевдокода для данного метода выглядит следующим образом:

In [ ]:
def transformer_block_with_ia3(x):
    residual = x
    x = ia3_self_attention(x)
    x = LN(x + residual)
    residual =  x
    x = x @ W_1  # FFN in
    x = l_ff * gelu(x)  # (IA)**3 scaling
    x = x @ W_2  # FFN out
    x = LN(x + residual)
    return x

def ia3_self_attention(x):
    k, q, v = x @ W_k, x @ W_q, x @ W_v
    k = l_k * k
    v = l_v * v
    return softmax(q @ k.T) @ v

Чтобы не переписывать практически целиком весь BERT и не мучаться с корректной инициализацией весов попробуем обучить этот метод в самом конце с использованием библиотеки peft

Примерный код для блока attention с использованием peft-метода IA^3 может выглядеть примерно так:

In [ ]:
class IA3Attention(nn.Module):
    def __init__(self, config: LLaMAConfig):
        super().__init__()
        self.config = config
        self.n_heads = config.n_heads
        self.head_dim = config.dim // config.n_heads

        self.q_proj = NoInitLinear(config.dim, config.dim, bias=False, dtype=config.dtype)
        self.k_proj = NoInitLinear(config.dim, config.dim, bias=False, dtype=config.dtype)
        self.v_proj = NoInitLinear(config.dim, config.dim, bias=False, dtype=config.dtype)
        self.o_proj = NoInitLinear(config.dim, config.dim, bias=False, dtype=config.dtype)

        # IA3-specific parameters:
        self.peft_l_k = nn.Parameter(torch.ones(1, self.n_heads, 1, self.head_dim, dtype=config.dtype))
        self.peft_l_v = nn.Parameter(torch.ones(1, self.n_heads, 1, self.head_dim, dtype=config.dtype))

    def forward(self, hidden_states, attention_mask):
        batch_size, q_seq_len, hidden_dim = hidden_states.size()

        # (batch_size, num_heads, q_seq_len, head_dim)
        query_states = self.q_proj(hidden_states).view(
            batch_size, q_seq_len, self.n_heads, self.head_dim).transpose(1, 2)
        key_states = self.k_proj(hidden_states).view(
            batch_size, q_seq_len, self.n_heads, self.head_dim).transpose(1, 2)
        value_states = self.v_proj(hidden_states).view(
            batch_size, q_seq_len, self.n_heads, self.head_dim).transpose(1, 2)
        
        # IA3-specific:
        query_states = query_states * self.peft_l_k
        value_states = value_states * self.peft_l_v
        # end of IA3-specific

        scores = torch.matmul(
            query_states, key_states.transpose(3, 2).type_as(query_states) / math.sqrt(self.head_dim)
        )
        scores += attention_mask

        # (batch_size, num_heads, q_seq_len, kv_seq_len)
        attn_weights = F.softmax(scores.float(), dim=-1).type_as(scores)
        # (batch_size, num_heads, q_seq_len, head_dim)
        attn_output = torch.matmul(attn_weights, value_states.type_as(query_states))
        # (batch_size, q_seq_len, hidden_dim)
        attn_output = attn_output.transpose(1, 2).contiguous().view(
            batch_size, q_seq_len, hidden_dim,
        )
        attn_output = self.o_proj(attn_output)
        check_nan(attn_output)
        return {"attn_output": attn_output}

## Библиотека PEFT

PEFT (Parameter-Efficient Fine-Tuning) — это библиотека для эффективной тренировки больших предобученных моделей на различные прикладные задачи без необходимости обучения всех параметров модели, поскольку это чрезвычайно дорого. PEFT методы позволяют обучать лишь небольшое количество (дополнительных) параметров модели — значительно снижая вычислительные затраты и занимаемое место на диске — при этом он обеспечивают сопоставимую с полным fine-tunингом модели производительность. Это делает более доступным обучение и хранение больших языковых моделей (LLM) на потребительском оборудовании.

PEFT интегрирован с библиотеками Transformers, Diffusers и Accelerate для обеспечения более быстрого и удобного способа загрузки, обучения и использования больших моделей для вывода.

Давайте посмотрим как очень просто можно применить адаптеры к нашей предобученной модели ruConversationalBert

In [ ]:
from peft import LoraConfig, get_peft_model, IA3Config,  TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["attention.self.query", "attention.self.key", "attention.self.value","attention.output.dense"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)
model = AutoModelForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased-conversational", num_labels=len(id2label), id2label=id2label, label2id=label2id
).to(device)

peft_model = get_peft_model(model, lora_config)
print(sum(param.numel() for param in peft_model.parameters() if param.requires_grad), "trainable parameters in model")

In [ ]:
model

In [ ]:
best_valid_loss = float('inf')

optimizer = AdamW(peft_model.parameters(), lr = 1e-5)
cross_entropy = torch.nn.functional.binary_cross_entropy_with_logits
epochs = 3

train_losses=[]
valid_losses=[]

for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    (train_loss, _), model_embeder = train(peft_model)
    (valid_loss, _), model_embeder = evaluate(peft_model)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(peft_model.state_dict(), 'saved_weights.pt')
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

Теперь давайте посмотрим, как можно обучить модель с использоаванием метода IA^3  

In [ ]:
ia3_config = IA3Config(
    task_type=TaskType.SEQ_CLS,
    target_modules=["attention.self.query", "attention.self.key", "attention.self.value", "intermediate.dense", "output.dense"],
    feedforward_modules=["intermediate.dense", "output.dense"]
)

model = AutoModelForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased-conversational", num_labels=len(id2label), id2label=id2label, label2id=label2id
).to(device)

peft_model = get_peft_model(model, ia3_config)
print(sum(param.numel() for param in peft_model.parameters() if param.requires_grad), "trainable parameters in model")

In [ ]:
best_valid_loss = float('inf')

optimizer = AdamW(peft_model.parameters(), lr = 1e-5)
cross_entropy = torch.nn.functional.binary_cross_entropy_with_logits
epochs = 3

train_losses=[]
valid_losses=[]

for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
`
    (train_loss, _), model_embeder = train(peft_model)
    (valid_loss, _), model_embeder = evaluate(peft_model)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(peft_model.state_dict(), 'saved_weights.pt')
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

## Prefix-Tuning

[Prefix Tuning](https://arxiv.org/pdf/2101.00190.pdf) вводит новые параметры в блоки Multi-Head Attention каждого слоя Transformer. В иллюстрации ниже префиксы отмечены розовым и фиолетовым цветами. Более конкретно, мы добавляем обучаемые векторы префиксов PK и PV к ключам и значениям входа головы внимания, каждый с конфигурируемой длиной префикса l (атрибут prefix_length):

$$
\text{head}_i = \text{Attention}(QW_{i}^{Q}, [PK_{i}^{K}, WK_{i}^{K}], [PV_i^{V}, VW^{V}_{i}])
$$


![](./images/prefix.png)

Если коротко сформулировать принцип работы prefix-tuning`а, то мы делаем fine-tuning только части нашего входа модели - soft-prompt

в псевдокоде этот методе может выглядеть примерно так:

In [ ]:
def prompt_tuning_attention(input_ids):
    q = x @ W_q
    k = cat([s_k, x]) @ W_k  # prepending  soft-promt, that will be trained 
    v = cat([s_v, x]) @ W_v  # prepending  soft-promt, that will be trained

Обучаясь всего на 0,1% параметров, Prefix-tuning показывает сопоставимую производительность в условиях полного объема данных, превосходит FT в условиях недостатка данных и лучше экстраполируется на примеры с темами, не встречавшимися во время обучения.

Рассмотрим библиотеку для работы с prefix-tunig`ом от коллег из SberDevices под названием ruprompts. Она заточена под использование с немного устаревшей моделью rugpt-3, однако для быстрого прототипирования иногда может быть полезна

Общая идея следующая: поскольку все слова, а точнее токены, переводятся в эмбеддинги (векторы фиксированной размерности), то эмбеддинги, соответствующие затравке, можно напрямую обучить градиентным спуском.

Обучаемая затравка (trainable prompt) логично разбивается на два компонента: формат (prompt format) и провайдер (prompt provider). Поясним на примере. Допустим, мы хотим обучить нейросеть отвечать на вопрос после прочтения текста. В случае, если мы решаем задачу методом zero-shot, формат затравки, скорее всего, будет примерно таким:

```
Текст:
{passage}

Вопрос: {question}
Ответ:
```

Например, этот обучающий пример:

```
{
    "passage": "GPT-3 устроена следующим образом: [...]",
    "question": "Как устроен self-attention?"
}
```

будет отформатирован и подан в модель в следующем виде:


```
Текст:
GPT-3 устроена следующим образом: [...]

Вопрос: Как устроен self-attention?
Ответ:
```

Сгенерированные моделью следующие токены мы и будем считать ответом.

Если же мы не уверены в том, что текстовые инструкции `(Текст:\n, \nВопрос:, \nОтвет:)` достаточно хорошо подходят к задаче, то prompt tuning позволяет нам заменить их на обучаемые токены (`<P>`) и контролировать только их количество. Таким образом, формат затравки примет следующий вид:

```
<P><P><P><P>{passage}<P><P><P><P>{question}<P><P><P><P>
```

При переводе токенов в эмбеддинги вместо словарных токенов подставляются их обычные эмбеддинги, а вместо обучаемых токенов (`<P>`) последовательно подставляются дифференцируемые эмбеддинги из провайдера

Эта библиотека обладает большим количества предобученных промптов, затюненых под разные задачи и именно этот сценарий мы с вами и рассмотрим - посмотрим на доступный инструментарий этой библиотеки

In [165]:
!pip install ruprompts -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
# although not used directly, this import is necessary,
# since it adds custom pipelines to transformers
import ruprompts

from transformers import pipeline

In [9]:
ppln_joke = pipeline("text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_joke", device=0)

No model was supplied, defaulted to sberbank-ai/rugpt3large_based_on_gpt2 and revision aa2b602 (https://huggingface.co/sberbank-ai/rugpt3large_based_on_gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [10]:
ppln_joke("Говорит как-то дуб вороне")

[{'generated_text': 'Говорит как-то дуб вороне. Стоит вороне, говорит дуб. А потом так хитро говорит дубу:- Ворона, я тебе обещал... Это слово в вороне и застряло.'}]

Конечно удобнее не указывать аргумент модели, однако это приводит к повторному созданию одной и той же модели каждый раз, когда мы создаем конвейер. Простое решение - создать модель и токенизатор один раз, а затем передавать их всем конвейерам:

In [11]:
from transformers import GPT2LMHeadModel, AutoTokenizer

model_id = "sberbank-ai/rugpt3large_based_on_gpt2"
model = GPT2LMHeadModel.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [12]:
ppln_proverb = pipeline("text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_proverb", model=model, tokenizer=tokenizer, device=0)
ppln_proverb("Сколько ни")

prompt.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

prompt_provider.bin:   0%|          | 0.00/369k [00:00<?, ?B/s]

[{'generated_text': 'Сколько ни спорь, а в ум войдешь.'}]

In [15]:
long_text = """Несмотря на споры, большинство учёных сошлись во мнении, что кошка является полуодомашненным животным, то есть она способна на сосуществование с человеком, но, потеряв с ним контакт, легко возвращаются к дикому образу существования. Хотя у кошки наблюдаются генетические изменения в сравнении с диким предком, эта разница в 10 раз меньше, чем у собак с волками. Учёные считают, что дикая кошка действительно могла сама прийти к человеку, чтобы питаться грызунами, а такие отношения характеризовались как соседские, и уже через несколько тысяч лет люди сами стали одомашнивать маленьких хищников. Это также, вероятно, объясняет, почему модель поведения кошки почти не изменилась; при одомашнивании собаки из волка человек изменил её образ жизни и среду обитания, кошка же претерпела минимальные изменения Кошка сумела сохранить модель поведения, присущую её диким предкам. Она почти так же хорошо охотится, как дикая кошка, но в то же время способна мирно сосуществовать с человеком, проявлять к нему эмоциональную привязанность, нежность или даже выказывать игривое поведение."""
ppln_summary = pipeline("text2text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_summarization_mlsum", model=model, tokenizer=tokenizer, device=0)

ppln_summary(long_text, num_beams=5, num_return_sequences=1)

[{'generated_text': 'Учёные считают, что дикая кошка могла сама прийти к человеку, чтобы питаться грызунами'}]

In [16]:
ppln_detox = pipeline("text2text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_detox_russe", model=model, tokenizer=tokenizer, device=0)
ppln_detox({"toxic_comment": "Ублюдок, мать твою, а ну иди сюда"})

prompt.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

prompt_provider.bin:   0%|          | 0.00/738k [00:00<?, ?B/s]

[{'generated_text': 'Попроси его зайти сюда'}]

In [17]:
context = """В 1997 году Шмидхубер и Сепп Хохрайтер опубликовали работу, описывающую рекуррентную нейронную сеть, которую авторы назвали «Долгая краткосрочная память». В 2015 году эта архитектура была использована в новой реализации распознавания речи в программном обеспечении компании Google для смартфонов.

Исследования Шмидхубера также включают в себя генерализации колмогоровской сложности и метрики «скорость важна» (Speed Prior), создание концепции Машины Гёделя.

В 2014 году Шмидхубер основал компанию Nnaisense для работы в сфере коммерческого применения технологий искусственного интеллекта в таких областях как финансы, тяжёлая промышленность и самоуправляемый автотранспорт. Сепп Хохрайтер и Яан Таллинн занимают в компании пост советников."""

ppln_qa = pipeline("text2text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_qa_sberquad", model=model, tokenizer=tokenizer, device=0)
ppln_qa({"context": context, "question": "С кем Шмидхубер опубликовал работу?"})

prompt.json:   0%|          | 0.00/549 [00:00<?, ?B/s]

prompt_provider.bin:   0%|          | 0.00/369k [00:00<?, ?B/s]

[{'generated_text': 'Сепп Хохрайтер'}]